This workflow follows one from the Fierer lab, which can be found here:
https://github.com/amoliverio/data_processing/blob/master/data_processing_tutorial.md

In [5]:
pwd

'/Users/mckenzielab/Desktop/Alex'

## activate qiime environment

In [ ]:
macqiime

MacQIIME 1.9.1-2015

In [6]:
!validate_mapping_file.py -m DevelopmentalBorealToad_metadata.txt -o mapcheck/

Errors and/or warnings detected in mapping file.  Please check the log and html file for details.


In [ ]:
ssh aalexiev@microbe.colorado.edu

### how many instances of the ITS forward primer occur in the sequence file - to determine if I need to cut the adapters off

In [6]:
grep -o 'TTACTTCCTCTAAATGACCAAGCCGTGTAGATCTCGGTGGTCGCCGTATCATT' \
BTP_ITS_seqs/Undetermined_S0_L001_R1_001.fastq | wc -l
#this output on the server was zero

SyntaxError: invalid syntax (<ipython-input-6-dbea881ce6e1>, line 1)

# demultiplex sequences (did not find any adapter seqs in seq files)

In [ ]:
prep_fastq_for_uparse_paired.py -i BTP_ITS_seqs/Undetermined_S0_L001_R1_001.fastq.gz \
-r BTP_ITS_seqs/Undetermined_S0_L001_R2_001.fastq.gz \
-b BTP_ITS_seqs/Undetermined_S0_L001_I1_001.fastq.gz \
-m DevelopmentalBorealToad_metadata.txt \
-o demultiplexed_seqs \
-c

prep_fastq_for_uparse_paired.py is a python script written by Jonathan Leff (https://github.com/leffj/helper-code-for-uparse/blob/master/prep_fastq_for_uparse_paired.py). It preps raw paired end sequences in the fastq.gz file from the sequencing facility and demultiplexes them for later input into uparse. -i is the sequence reads in fastq format. -r is the reverse reads in fastq format. -b is the barcode/index reads in fastq. -m is the mapping file for the sequences in txt format, with the barcodes in the second column. -o the output file directory where you will get your demultiplexed seqs. -c has the default of FALSE, make TRUE if you need to reverse compliment your sequences before demultiplexing (depends on what primers you used). All these were run on one lane (L001 part of the file name).

In [ ]:
#useful code from troubleshooting low demultiplexing number
#convert fastq from demultiplexing to fasta
#sed -n '1~4s/^@/>/p;2~4p' ../demultiplexed_seqs/demultiplexed_seqs_2.fq > OUTFILE.fasta
#compress a file
#tar zcvf seqstocompare.tar.gz seqstocompare

In [ ]:
grep -c '+' demultiplexed_seqs/demultiplexed_seqs_1.fq
#9357573 + 1 = 9357574 total reads across all samples

Count all instances of + in the demultiplexed sequence file, which gives you the total reads across all the samples - 1, since the format is SEQ + SEQ + SEQ in the file.

## Merge paired reads

In [ ]:
#download usearch 8, 32 bit version and fix permissions
#chmod a+rx /Users/mckenzielab/usearch8
#can't use 32 bit version with my large dataset, so going to use Noah Fierer's microbe server, which has usearch 8

In [ ]:
usearch8 -fastq_mergepairs demultiplexed_seqs/demultiplexed_seqs_1.fq \
-reverse demultiplexed_seqs/demultiplexed_seqs_2.fq -fastq_minovlen 16 \
-fastq_minmergelen 200 -fastqout demultiplexed_seqs/demultiplexed_seqs_merged.fq -notrunclabels \
-report demultiplexed_seqs/merge_rpt.txt

Usearch fastq_mergepairs script merges the sequences and forms a consensus sequence if there is mismatch in the overlap region. The reverse read is reverse complimented so it matches with the forward read when being aligned. -fastq_mergepairs is the forward sequence file; -reverse is the reverse sequence file to be merged (in paired end seq); -fastq_minovlen Discard pair if alignment is shorter than given value. Default 16; -fastq_minmergelen Minimum length for the merged sequence. This gets rid of artifacts (i.e. things that merged that are less than 200 seqs are probably sequencing artifacts like primers binding to each other and whatnot); -fastqout is the output directory; -notrunclabels don't truncate labels (I assume, this was included in the example I got from the Fierer lab but isn't in the current documentation for usearch); -report filepath for report

usearch v8.1.1861_i86linux64, 264Gb RAM, 32 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: jonathan.leff@colorado.edu

In [ ]:
less demultiplexed_seqs/merge_rpt.txt
q #to escape

Lots of "alignments with zero differences (mismatches)" - these are things that were great matches. Many had too many differences/mismatches to align.

In [ ]:
#Trim forward then reverse primer
cutadapt -g ^fwd_primer demultiplexed_seqs/demultiplexed_seqs_merged.fq > demultiplexed_seqs_merged_trimmedf.fq
cutadapt -a rev_primer demultiplexed_seqs_merged_trimmedf.fq > demultiplexed_seqs_merged_trimmedfr.fq

cutadapt cuts off the adapters, although this step wasn't necessary for me because the adapters were trimmed in the previous step with usearch. This is why it says "trimmed: 0" in the output here.

=== Summary ===

Total reads processed:               5,879,371
Reads with adapters:                         0 (0.0%)
Reads written (passing filters):     5,879,371 (100.0%)

Total basepairs processed: 1,427,153,452 bp
Total written (filtered):  1,427,153,452 bp (100.0%)

=== Adapter 1 ===

Sequence: FWD_PRIMER; Type: anchored 5'; Length: 10; Trimmed: 0 times.

=== Summary ===

Total reads processed:               5,879,371
Reads with adapters:                         0 (0.0%)
Reads written (passing filters):     5,879,371 (100.0%)

Total basepairs processed: 1,427,153,452 bp
Total written (filtered):  1,427,153,452 bp (100.0%)

=== Adapter 1 ===

Sequence: REV_PRIMER; Type: regular 3'; Length: 10; Trimmed: 0 times.

In [ ]:
#quality check sequences
usearch10 -fastq_eestats2 demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-output demultiplexed_seqs/demultiplexed_seqs_merged_eestats2.log -length_cutoffs 200,300,10
less demultiplexed_seqs/demultiplexed_seqs_merged_eestats2.log

Shows how many reads will pass an expected error filter at different length thresholds. Use this to determine where the cutoff should be, since I want to keep as many sequences as possible but not all will be useful. -length_cutoffs specifies the shortest length, then longest, and by what increment. Since the region I'm targeting is 230 bp long, the range 200-300 should capture that and all useful variants, and the cutoffs are going by 10. Expected error is calculated using the Phred scores at each bp to find the expected error for the whole read.

In [ ]:
#quality filtering step
usearch10 -fastq_filter demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-fastaout seqs_filt.fa -fastq_maxee 1.0 -fastq_trunclen 210

Picked 210 as the length to truncate at because that is where I'd still retain as many sequences as possible without getting too many bad quality reads based on EE. Chose 1% as the max EE score, so the expected error cutoff is 1% of sequence error probability (so at about 200 seqs for my amplicon, maybe 2 nt are wrong). 

In [ ]:
#dereplicate samples
usearch10 -fastx_uniques seqs_filt.fa -fastaout uniques.fa -sizeout -relabel Uniq

Derepilcation refers to finding only the unique sequences and using that consensus for finding abundances later.

## Make OTUs, cluster at 97% similarity

In [ ]:
#create OTUs
usearch10 -cluster_otus uniques.fa -otus rep_set97.fa -relabel 'OTU_'

usearch10 -usearch_global rep_set97.fa \
-db UNITE_db_s_7.2/sh_refs_qiime_ver7_97_s_01.12.2017.fasta \
-id 0.75 -strand both -matched rep_set97_filt.fa

Cluster_otus creates OTU's de novo. Chimeras are filtered. Unoise3 also removes reads with sequencing error (when identified by the algorithm, some may have slipped through obviously) [see zOTUs in next section]. Usearch_global searches for one hit to the database, in this case I am using UNITE since it is vetted and recommended by a number of fungal genomics papers. -id give the factional identity for matching OTU clusters (or in this case, centroid sequence for the cluster) to the database.

In [ ]:
#map demultiplexed reads to de novo database and build OTU table
usearch10 -otutab demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-otus rep_set97_filt.fa -otutabout otutab97.txt -mapout map.txt

This command maps the reads back to the OTUs they've been assigned and creates an OTU table from it.

## Add taxonomy, filter for chloro/mito/rare (artefacts), etc.

In [ ]:
#add taxonomy
biom convert -i otutab97.txt -o otutab97.biom --table-type 'OTU table' --to-json

assign_taxonomy.py -m rdp -i rep_set97_filt.fa -o rdp_assigned_taxonomy_otus \
-c 0.5 -r UNITE_db_s_7.2/sh_refs_qiime_ver7_97_s_01.12.2017.fasta \
-t UNITE_db_s_7.2/sh_taxonomy_qiime_ver7_97_s_01.12.2017.txt --rdp_max_memory 10000

biom add-metadata -i otutab97.biom --observation-metadata-fp \
rdp_assigned_taxonomy_otus/rep_set97_filt_tax_assignments.txt --sc-separated taxonomy \
--observation-header OTUID,taxonomy -o otutab97filt_wTax.biom

#remove chloroplasts and mitochondria
filter_taxa_from_otu_table.py \
-i otutab97filt_wTax.biom \
-o otutab_wTax_noChloroMito.biom \
-n c__Chloroplast,f__mitochondria

#remove OTU's that only occur once
filter_otus_from_otu_table.py \
-i otutab_wTax_noChloroMito.biom \
-o otutab_wTax_noChloroMitoSingl.biom \
-n 2

#convert biom table to text
biom convert -i otutab_wTax_noChloroMitoSingl.biom \
-o otutab_wTax_noChloroMitoSingl.txt --to-tsv --header-key=taxonomy

#otu table stats
biom summarize-table -i otutab_wTax_noChloroMitoSingl.txt \
-o otutab_wTax_noChloroMitoSingl_smry.txt
less otutab_wTax_noChloroMitoSingl_smry.txt #1030

## Make zOTUs for later comparison

In [ ]:
#usearch10 -unoise3 uniques.fa -zotus rep_set_zotus.fa -tabbedout unoise3.txt

#usearch10 -usearch_global rep_set_zotus.fa \
#-db /db_files/UNITE_files/sh_qiime_release_s_02.03.2015/sh_refs_qiime_ver7_dynamic_s_02.03.2015.fasta \
#-id 0.75 -strand both -matched rep_set_zotus_filt.fa

#usearch10 -fastx_relabel rep_set_zotus_filt.fa -prefix 'OTU_' -fastaout rep_set_zotus_filt_relabeled.fa -keep_annots

#usearch10 -otutab demultiplexed_seqs/demultiplexed_seqs_merged.fq -zotus rep_set_zotus_filt_relabeled.fa \
#-otutabout zotutab.txt -mapout zmap.txt

In [ ]:
#Add taxonomy, etc.

#biom convert -i zotutab.txt -o zotutab.biom --table-type 'OTU table' --to-json

#assign_taxonomy.py -m rdp -i rep_set_zotus_filt_relabeled.fa -o rdp_assigned_taxonomy_zotus -c 0.5 \
#-t /db_files/UNITE_files/sh_qiime_release_s_02.03.2015/sh_taxonomy_qiime_ver7_97_s_02.03.2015.txt \
#-r /db_files/UNITE_files/sh_qiime_release_s_02.03.2015/sh_refs_qiime_ver7_dynamic_s_02.03.2015.fasta \
#--rdp_max_memory 10000

#biom add-metadata -i zotutab.biom --observation-metadata-fp \
#rdp_assigned_taxonomy_zotus/rep_set_zotus_filt_relabeled_tax_assignments.txt --sc-separated taxonomy \
#--observation-header OTUID,taxonomy -o zotutab_wTax.biom

#filter_taxa_from_otu_table.py -i zotutab_wTax.biom \
#-o zotutab_wTax_noChloroMito.biom -n c__Chloroplast,f__mitochondria

#biom convert -i zotutab_wTax_noChloroMito.biom \
#-o zotutab_wTax_noChloroMito.txt --to-tsv --header-key=taxonomy

## Beta diversity script to get UniFrac

In [ ]:
make_phylogeny.py -i ../rep_set97_filt.fa \
-t fasttree -o 97otus_repicked.tre -l tree_log_file

In [ ]:
biom convert -i input2_otu.txt \
-o input2_OUT.biom \
--table-type="OTU table" --to-json

In [ ]:
core_diversity_analyses.py -i input2_OUT.biom \
-o fungdiv_UF \
-m ../DevelopmentalBorealToad_metadata.txt \
-t 97otus_repicked.tre \
-e 1030 --recover_from_failure

In [ ]:
#move files from microbe server to local machine
scp -r aalexiev@microbe.colorado.edu:FungalDevData_V0/BetaDiv .

# SourceTracker

SourceTracker uses a Gibbs sampler to determine whether a pre-defined source could account for some proportion of the population found in a pre-defined sink. It requires a column in the mapping file called SourceSink that has either "source" or "sink" for each sample, as well as an Env column. This is being run on my local machine.

In [ ]:
#WHOLE DATASET FOR LIFEENV FROM R
#convert text tables from R into biom tables
#had to use nano to add "#OTU_ids \t" to the beginning of the text file
source activate st2

biom convert -i lifeenv_OTU.txt \
-o lifeenv_OTU.biom \
--table-type="OTU table" --to-json

sourcetracker2 gibbs -i lifeenv_OTU.biom \
-m lifeenv_map.txt \
-o SourceTracker_results \
--burnin 100 

In [ ]:
#JUST 4 MILE SITE
biom convert -i mile_OTU.txt \
-o mile_OTU.biom \
--table-type="OTU table" --to-json

sourcetracker2 gibbs -i mile_OTU.biom \
-m mile_map.txt \
-o SourceTracker_fourmile \
--burnin 100

In [ ]:
#JUST adult and egg samples
biom convert -i adult_egg_OTU.txt \
-o adult_egg_OTU.biom \
--table-type="OTU table" --to-json

sourcetracker2 gibbs -i adult_egg_OTU.biom \
-m adult_egg_map.txt \
-o SourceTracker_adult_egg \
--burnin 100

# Pick OTU's with "antifungal amphibian bacteria" database

In [ ]:
#Check how many instances of the bacterial forward primer are in my file
cd Bacterial_OTUs

grep -o 'ATTAGAWACCCBDGTAGTCCGGCTGACTGACT' \
BTP_HOP_WORM_Miseq/Undetermined_S0_L001_R1_001.fastq.gz | wc -l
#this output was zero, so I don't need to trim adapters


#demultiplex sequences
prep_fastq_for_uparse_paired.py -i BTP_HOP_WORM_Miseq/Undetermined_S0_L001_R1_001.fastq.gz \
-r BTP_HOP_WORM_Miseq/Undetermined_S0_L001_R2_001.fastq.gz \
-b BTP_HOP_WORM_Miseq/Undetermined_S0_L001_I1_001.fastq.gz \
-m BTP_HOP_WORM_Miseq/DevelopmentalBorealToad_16Smetadata.txt \
-o demultiplexed_seqs
#This was a mixed run with other projects' data so I don't expect the whole thing to demultiplex
#Start time: 2019-07-03 15:25:23
#Seqs processed: 14711946 Percent kept:  49.0%
#End time: 2019-07-03 15:34:31


#merge paired reads then view report
usearch8 -fastq_mergepairs demultiplexed_seqs/demultiplexed_seqs_1.fq \
-reverse demultiplexed_seqs/demultiplexed_seqs_2.fq -fastq_minovlen 16 \
-fastq_minmergelen 200 -fastqout demultiplexed_seqs/demultiplexed_seqs_merged.fq -notrunclabels \
-report demultiplexed_seqs/merge_rpt.txt

#Merging
#  Fwd demultiplexed_seqs/demultiplexed_seqs_1.fq
#  Rev demultiplexed_seqs/demultiplexed_seqs_2.fq
#  Keep read labels
#00:37 2.5Gb  100.0% 75.2% merged

less demultiplexed_seqs/merge_rpt.txt
q #to escape

#Totals
#   7214311  Pairs (7.2M)
#   5426891  Merged (5.4M, 75.22%)
#   3901522  Alignments with zero diffs (54.08%)
#   1268481  Too many diffs (> 5) (17.58%)
#         0  Fwd tails Q <= 2 trimmed (0.00%)
#    134727  Rev tails Q <= 2 trimmed (1.87%)
#         0  Fwd too short (< 64) after tail trimming (0.00%)
#     33541  Rev too short (< 64) after tail trimming (0.46%)
#    471832  No alignment found (6.54%)
#         0  Alignment too short (< 16) (0.00%)
#     13566  Merged too short (< 200)
#      4471  Staggered pairs (0.06%) merged & trimmed
#     49.10  Mean alignment length
#    252.86  Mean merged read length
#      0.29  Mean fwd expected errors
#      0.31  Mean rev expected errors
#      0.30  Mean merged expected errors

#trim forward and reverse primers
#Trim forward then reverse primer
cutadapt -g ^fwd_primer demultiplexed_seqs/demultiplexed_seqs_merged.fq > demultiplexed_seqs_merged_trimmedf.fq
cutadapt -a rev_primer demultiplexed_seqs_merged_trimmedf.fq > demultiplexed_seqs_merged_trimmedfr.fq
#trimmed successfully

#check seq quality, filter based on this quality info
usearch10 -fastq_eestats2 demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-output demultiplexed_seqs/demultiplexed_seqs_merged_eestats2.log -length_cutoffs 200,300,10
less demultiplexed_seqs/demultiplexed_seqs_merged_eestats2.log
#found that maxee of 1.0 and trunclength of 250 is best
usearch10 -fastq_filter demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-fastaout seqs_filt.fa -fastq_maxee 1.0 -fastq_trunclen 250

#dereplicate samples
usearch10 -fastx_uniques seqs_filt.fa -fastaout uniques.fa -sizeout -relabel Uniq

In [ ]:
#pick 97% OTUs using anti-Bd bacteria database file
usearch10 -cluster_otus uniques.fa -otus rep_set97.fa -relabel 'OTU_'
#04:17 145Mb   100.0% 23892 OTUs, 19962 chimeras

usearch10 -usearch_global rep_set97.fa \
-db Bd_inhibitor_database/Amphibian-skin_bacteria_16S_sequences.fna \
-id 0.75 -strand both -matched rep_set97_filt.fa
#00:00 37Mb      0.1% Reading Bd_inhibitor_database/Amphibian-skin_bacteria_16S_s00:00 45Mb    100.0% Reading Bd_inhibitor_database/Amphibian-skin_bacteria_16S_sequences.fna
#00:00 11Mb      0.1% Masking (fastnucleo)                                       00:00 11Mb    100.0% Masking (fastnucleo)
#00:00 12Mb    100.0% Word stats          
#00:00 12Mb    100.0% Alloc rows
#00:00 20Mb    100.0% Build index
#00:00 54Mb   CPU has 32 cores, defaulting to 10 threads
#00:09 764Mb   100.0% Searching, 75.9% matched

#map demiltiplexed reads to de novo database and build OTU table
usearch10 -otutab demultiplexed_seqs/demultiplexed_seqs_merged.fq \
-otus rep_set97_filt.fa -otutabout otutab97.txt -mapout map.txt
#00:00 48Mb    100.0% Reading rep_set97_filt.fa
#00:00 14Mb    100.0% Masking (fastnucleo)     
#00:00 16Mb    100.0% Word stats          
#00:00 16Mb    100.0% Alloc rows
#00:00 33Mb    100.0% Build index
#00:00 66Mb   CPU has 32 cores, defaulting to 10 threads
#12:14 777Mb   100.0% Searching demultiplexed_seqs_merged.fq, 87.0% matched
#4722892 / 5426891 mapped to OTUs (87.0%)                                  
#12:14 777Mb  Writing otutab97.txt
#12:14 777Mb  Writing otutab97.txt ...done.

#add taxonomy
biom convert -i otutab97.txt -o otutab97.biom --table-type 'OTU table' --to-json

assign_taxonomy.py -m rdp -i rep_set97_filt.fa -o rdp_assigned_taxonomy_otus \
-c 0.5 -r Bd_inhibitor_database/Amphibian-skin_bacteria_16S_sequences.fna \
-t Bd_inhibitor_database/inhbit_UCLUST.txt --rdp_max_memory 10000

biom add-metadata -i otutab97.biom --observation-metadata-fp \
rdp_assigned_taxonomy_otus/rep_set97_filt_tax_assignments.txt --sc-separated taxonomy \
--observation-header OTUID,taxonomy -o otutab97filt_wTax.biom

#remove OTU's that only occur once
filter_otus_from_otu_table.py \
-i otutab97filt_wTax.biom \
-o otutab97_wTax_noSingl.biom \
-n 2

#convert biom table to text
biom convert -i otutab97_wTax_noSingl.biom \
-o otutab97_wTax_noSingl.txt --to-tsv --header-key=taxonomy

#otu table stats
biom summarize-table -i otutab97_wTax_noSingl.txt \
-o otutab97_wTax_noSingl_smry.txt
less otutab97_wTax_noSingl_smry.txt #ok to rarefy to 1030 to match fungal data